# Basic Mixed Hot Water Cylinder

> Simple (fast to run) model of a mixed hot water cylinder

In [ ]:
#| default_exp core

In [1]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
from fastcore.utils import *
import pandas as pd
from cylinder.demand import *
from cylinder.power import *
import matplotlib.pyplot as plt
import random
from collections import deque
import numpy as np

ModuleNotFoundError: No module named 'cylinder'

In [3]:
class HWC():
  """
  Simple mixed model for a hot water cylinder
  """
  # Define constants for clearer code
  def __init__(self,
              element = 3000, # Watts
              T_set = 65, # °C
              T_deadband = 2, # °C
              radius =.25,  # Meters
              height = 1, # Meters
              unit=None,  # Unit of measure for volume
              bedrooms=None,
              rtp = None,
              id = None):
    super(HWC, self).__init__()

    self.T_ambient = 15 #  Air temperature that the cylinder located in °C
    self.T_cold = 15 # Make up water temperature °C
    self.T_demand = 45 #T_demand - temperature of the end use (shower)  °C
    self.T_deadband = T_deadband #T_deadband  - thermostat deadband °C
    self.T_set = T_set #T_set - thermostat set point °C
    # self.T_std = 45
    self.U = 0.5/60/1000 
    self.Cv = 4.184 
    self.rho = 1000 
    self.Area = 2*np.pi*radius*(radius+height)
    self.Volume = np.pi*radius**2*height
    # self.unit = random.choice(range(10)) if unit is None else unit
    # self.bedrooms = random.choice(range(5))+1 if bedrooms is None else bedrooms
    self.Q = element/1000
    self.T_stat = False
    self.rtp = rtp
    self.id = id
    # self.thermogram = np.ones([7,24])*.5
    self.alpha = 0.01
    self.action_space = spaces.Discrete(2)
    self.observation_space = spaces.Box(low=-1, 
                                        high=1, 
                                        shape=(5,), 
                                        dtype=np.float32)
    self.current_step = 0

      # spaces.Box(low=0, high=255,
      #                                   shape=(N_CHANNELS, HEIGHT, WIDTH), 
      #                                   dtype=np.uint8)

NameError: name 'gym' is not defined

In [4]:
@patch
def step(self:HWC, action):
    self._thermostat()
    self.action = action
    if action == 1:
        self.last_power = self.current_step
    # print(action)
    # import pdb; pdb.set_trace()
    flow = self.flow_df.iloc[self.current_step]
    self.flow = flow*(self.T_demand-self.T_cold)/(self.T-self.T_cold)
    self.T  = self._update()

    r,c = self.rtp.iloc[self.current_step].name.dayofweek, self.rtp.iloc[self.current_step].name.hour
    self.thermogram[r,c] = self.T_stat * self.action * self.Q /60 +self.thermogram[r,c]*(1- self.alpha)
    self.power_hist.append(self.T_stat * self.action * self.Q /60)
    # self.T_history.append(self.T)
    # self.history.append([self.id,
    #                      self.df.iloc[self.current_step].name.hour,
    #                      self.df.iloc[self.current_step].name,
    #                      action*self.T_stat,
    #                      self.T,
    #                      self.flow])
    # reward, done = self._get_reward()
    self.current_step += 1

    # Are we at the left of the grid?
    done = False
    # Null reward everywhere except when reaching the goal (left of the grid)
    reward = self._time_reward(self.time_since_full,2)
    # reward += self._time_reward((self.current_step-self.last_power),40)
    # print(self.time_since_full,1-self.time_since_full/30)
    # reward += np.array(self.cost_hist).sum()/6
    # reward += 5*self._power_reward(self.diff)
    # reward = 0
    reward += 2*self.rtp.iloc[self.current_step].price_indicator*self.action
    # reward = self._power_reward(self.rtp.iloc[self.current_step].cost*self.action)
    # print(reward,(self.current_step-self.last_power),self._time_reward(self.time_since_full),self._power_reward(self.diff),2*self._cost_reward(self.rtp.iloc[self.current_step].cost)*self.action)
    # print(reward,self.diff)
    # reward -= .05*(self.T_set - self.T)
    # print(reward)
    if self.T <= 50:
        done = True
        reward = -100
    # done = False if self.T >= 45 else True    # Optionally we can pass additional info, we are not using that for now
    # done = False if self.current_step <= 60*24*20 else True    # Optionally we can pass additional info, we are not using that for now

    info = self._get_info()
    observation = self._get_observation()
    return observation, reward, done, info

@patch
def _thermostat(self:HWC):
    if (self.T < self.T_set- self.T_deadband) :
        self.T_stat = 1
        # print(self.current_step, 't on')
    elif (self.T > self.T_set) :
        self.T_stat = 0
        self.full_charge = self.current_step
        # print(self.current_step, 't off')

    return 

@patch
def _power_reward(self:HWC,x): return .5-pow(x/5,2)
@patch
def _time_reward(self:HWC,x, b=2): return -pow(x/b,2) #return 2 - 1.5**(x/120)
@patch
def _cost_reward(self:HWC,x): return .1-np.exp(-30*x)#2 - 1.5**(x)
@patch
def _update(self:HWC, ts = 60):
    # if self.T_stat * self.action == 1 :
    #     print(self.action,'heating') 
    return self.T + \
           self.flow/self.Volume*(self.T_cold-self.T) * ts + \
           self.T_stat * self.action * self.Q / (self.Cv*self.rho*self.Volume) * ts - \
           (self.U*self.Area)/(self.Cv*self.rho*self.Volume)*(self.T-self.T_ambient) * ts

@patch
def _get_info(self:HWC):
    return {
        "temperature": self.T
    }

@patch
def _get_observation(self:HWC):
    d,h = self.rtp.iloc[self.current_step].name.dayofweek, self.rtp.iloc[self.current_step].name.hour
    load_hist = np.roll(self.thermogram.flatten(), -(d*24+h+3))[-12:]
    self.recent_hist = np.array([np.array(self.power_hist)[i:i+60].sum() for i in range(0, len(self.power_hist), 60)][::-1])
    last_power = self.current_step-self.last_power
    self.diff = self.recent_hist.sum()- load_hist.sum()

    # print(recent_hist, load_hist, self.diff)
    self.time_since_full = (self.current_step - self.full_charge)/(60)
    
    if self.current_step == 0.:
        thermo_info = int(self.T_stat)
    elif self.action == 1.:
        thermo_info = int(self.T_stat)
    else: 
        thermo_info = -1. #no information

    # print(thermo_info)
    # np.array(self.thermo_hist) # action = 1 and thermostat = 1 / action = 0 and no thermostat feedback / action = 1 and thermostat = 0
    obs = np.concatenate((np.array(thermo_info),
    self.time_since_full,
    self.diff,
    last_power/60,
                        #   load_hist,
                        #  recent_hist,
                          np.array(self.rtp.iloc[self.current_step]['n_cost'],self.rtp.iloc[self.current_step]['price_indicator'])), 
                          axis=None, 
                          dtype=np.float32)
    print(obs)
    # for i in range(6):
    #     self.thermo_hist.append(self.T_stat)
    #     self.cost_hist.append(self.T_stat*self.Q)
    # print(np.array([xi for xi in bb]))
    return obs
    # return {"temperature": self.T,
    #         "thermostat": self.T_stat}

@patch
def reset(self:HWC):
    self.power_hist = deque(np.ones(60*12)*.5/60,maxlen=60*12)

    self.action = 0
    self.last_power = 0
    self.T = self.T_set + np.random.uniform(-4, 0)
    self.unit = random.choice(range(10)) 
    self.bedrooms = random.choice(range(5))+1 
    self.flow_df = load_demand(bed=self.bedrooms, unit=self.unit)
    self.thermogram = np.ones([7,24])*(.5+self.bedrooms*.1)
    self.full_charge = self.current_step
    self.thermo_hist = deque(np.ones(60*24)/(60*24),maxlen=60*24)
    self.cost_hist = deque(np.ones(60*24)*.01,maxlen=60*24)
    return self._get_observation()

NameError: name 'HWC' is not defined

In [5]:
#| hide
import nbdev; nbdev.nbdev_export()